In [70]:
import mediapipe as mp
import cv2
import os
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
import pickle

In [43]:
model = load_model('../models/cnn_lstm_model.h5')

In [44]:
# STEP 1: Define pose estimation model and functions
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.7, min_tracking_confidence=0.7)
mp_drawing = mp.solutions.drawing_utils

I0000 00:00:1715442663.877372       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2


In [45]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [46]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

In [47]:
def duplicate_frames(df, max_frames):
    num_dup = max_frames - len(df)

    if num_dup > 0 and num_dup < len(df):
        # Duplicate frames up to the required number of duplicates
        dup_frames = df.loc[:num_dup - 1].copy()
        df = pd.concat([df, dup_frames], ignore_index=True)
    elif num_dup > len(df):
        # Calculate the number of times to duplicate based on the original DataFrame length
        num_duplicates = (max_frames // len(df)) - 1
        orig_df_len = len(df)  # Store the original DataFrame length
        remainder = max_frames % len(df)  # Calculate the remainder
        for _ in range(num_duplicates):
            df = pd.concat([df, df[:orig_df_len]], ignore_index=True)  # Duplicate original rows

        # Add the remaining rows from the original DataFrame to fill up to max_frames
        df = pd.concat([df, df[:remainder]], ignore_index=True)

    return df

In [48]:
def calculate_angle(df, a, b, c):
    a_x = a + '_x'
    a_y = a + '_y'
    b_x = b + '_x'
    b_y = b + '_y'
    c_x = c + '_x'
    c_y = c + '_y'

    radians = np.arctan2(df[c_y].values - df[b_y].values, df[c_x].values - df[b_x].values) - \
              np.arctan2(df[a_y].values - df[b_y].values, df[a_x].values - df[b_x].values)
    angles = np.abs(radians * 180.0 / np.pi)

    # Adjust angles greater than 180 degrees
    angles = np.where(angles > 180.0, 360 - angles, angles)

    return angles

In [49]:
def calculate_shank_angle(df, ankle, knee):
    """
    Calculate the angle between the shank (knee to ankle) and the horizontal line.

    Args:
        df: DataFrame containing the relevant columns.
        ankle: Name of the ankle column.
        knee: Name of the knee column.

    Returns:
        The angle in degrees between the shank and the horizontal line.
    """
    ankle_x = ankle + '_x'
    ankle_y = ankle + '_y'
    knee_x = knee + '_x'
    knee_y = knee + '_y'

    angle_rad = np.arctan2(df[knee_y] - df[ankle_y], df[knee_x] - df[ankle_x])
    angle_deg = np.degrees(angle_rad) + 90
    return angle_deg

In [50]:
def calculate_lean_angle(df, a, b):
    a_x = a + '_x'
    a_y = a + '_y'
    b_x = b + '_x'
    b_y = b + '_y'

    radians = np.arctan2(0 - df[b_y].values, df[b_x].values - df[b_x].values) - \
              np.arctan2(df[a_y].values - df[b_y].values, df[a_x].values - df[b_x].values)
    angles = np.abs(radians * 180.0 / np.pi)

    # Adjust angles greater than 180 degrees
    angles = np.where(angles > 180.0, 360 - angles, angles)

    return angles

In [51]:
column_names = ['left_shoulder_x', 'left_shoulder_y', 'left_shoulder_z', 'left_shoulder_vis', 'right_shoulder_x', 'right_shoulder_y', 'right_shoulder_z', 'right_shoulder_vis', 'left_elbow_x', 'left_elbow_y', 'left_elbow_z', 'left_elbow_vis', 'right_elbow_x', 'right_elbow_y', 'right_elbow_z', 'right_elbow_vis', 'left_wrist_x', 'left_wrist_y', 'left_wrist_z', 'left_wrist_vis', 'right_wrist_x', 'right_wrist_y', 'right_wrist_z', 'right_wrist_vis', 'left_pinky_x', 'left_pinky_y', 'left_pinky_z', 'left_pinky_vis', 'right_pinky_x', 'right_pinky_y', 'right_pinky_z', 'right_pinky_vis', 'left_index_x', 'left_index_y', 'left_index_z', 'left_index_vis', 'right_index_x', 'right_index_y', 'right_index_z', 'right_index_vis', 'left_thumb_x', 'left_thumb_y', 'left_thumb_z', 'left_thumb_vis', 'right_thumb_x', 'right_thumb_y', 'right_thumb_z', 'right_thumb_vis', 'left_hip_x', 'left_hip_y', 'left_hip_z', 'left_hip_vis', 'right_hip_x', 'right_hip_y', 'right_hip_z', 'right_hip_vis', 'left_knee_x', 'left_knee_y', 'left_knee_z', 'left_knee_vis', 'right_knee_x', 'right_knee_y', 'right_knee_z', 'right_knee_vis', 'left_ankle_x', 'left_ankle_y', 'left_ankle_z', 'left_ankle_vis', 'right_ankle_x', 'right_ankle_y', 'right_ankle_z', 'right_ankle_vis', 'left_heel_x', 'left_heel_y', 'left_heel_z', 'left_heel_vis', 'right_heel_x', 'right_heel_y', 'right_heel_z', 'right_heel_vis', 'left_foot_index_x', 'left_foot_index_y', 'left_foot_index_z', 'left_foot_index_vis', 'right_foot_index_x', 'right_foot_index_y', 'right_foot_index_z', 'right_foot_index_vis']
print(column_names)

['left_shoulder_x', 'left_shoulder_y', 'left_shoulder_z', 'left_shoulder_vis', 'right_shoulder_x', 'right_shoulder_y', 'right_shoulder_z', 'right_shoulder_vis', 'left_elbow_x', 'left_elbow_y', 'left_elbow_z', 'left_elbow_vis', 'right_elbow_x', 'right_elbow_y', 'right_elbow_z', 'right_elbow_vis', 'left_wrist_x', 'left_wrist_y', 'left_wrist_z', 'left_wrist_vis', 'right_wrist_x', 'right_wrist_y', 'right_wrist_z', 'right_wrist_vis', 'left_pinky_x', 'left_pinky_y', 'left_pinky_z', 'left_pinky_vis', 'right_pinky_x', 'right_pinky_y', 'right_pinky_z', 'right_pinky_vis', 'left_index_x', 'left_index_y', 'left_index_z', 'left_index_vis', 'right_index_x', 'right_index_y', 'right_index_z', 'right_index_vis', 'left_thumb_x', 'left_thumb_y', 'left_thumb_z', 'left_thumb_vis', 'right_thumb_x', 'right_thumb_y', 'right_thumb_z', 'right_thumb_vis', 'left_hip_x', 'left_hip_y', 'left_hip_z', 'left_hip_vis', 'right_hip_x', 'right_hip_y', 'right_hip_z', 'right_hip_vis', 'left_knee_x', 'left_knee_y', 'left_kne

In [52]:
# Create an opencv object from our video
cap = cv2.VideoCapture('test.mp4')

# Get the total number of frames in the video
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Create features list
features = []

# Set mediapipe model
while cap.isOpened():
    # Read feed
    ret, frame = cap.read()
    # Once feed is at its last frame, there will be no ret value, break loop and move to next video
    if not ret:
        break

    # Make detections using previously defined function to detect keypoints
    image, results = mediapipe_detection(frame, pose)

    # Export Keypoints
    keypoints = results.pose_landmarks.landmark # Extract keypoints from the results object

    # Remove facial landmarks
    keypoints_noface = keypoints[11:]
    
    # flatten landmarks_noface for each frame
    for keypoint in keypoints_noface:
        # Create a list of features using the extracted coordinates for each frame
        feature_list = [value for keypoint in keypoints_noface for value in [keypoint.x, keypoint.y, keypoint.z, keypoint.visibility]]
    
    # Append each frame's list of coordinates into the empty features list
    features.append(feature_list)

# Create a dataframe with the features list
df = pd.DataFrame(features, columns=column_names)

# Duplicate frames to match the required dataframe shape
df_dup = duplicate_frames(df, 258)

    
#     # # Save landmark of each frame as a list
#     # features.append(landmarks)

#     # # Save numpy in new folder
#     # timestep+=1# Save the keypoints data in a numpy array onto the path defined
    
#     # # Show to screen with defined function to draw points and lines
#     # draw_landmarks(image, results)
#     # cv2.imshow('OpenCV Feed', image)

#     # Break loop if 'q' is pressed
#     if cv2.waitKey(10) & 0xFF == ord('q'):
#         break

# # Release video capture and destroy OpenCV windows
# cap.release()
# if cv2.getWindowProperty('OpenCV Feed', cv2.WND_PROP_VISIBLE) >= 1:
#     cv2.destroyAllWindows()
# cv2.waitKey(1)  # Added this
# cap.release

In [69]:
df_dup.head()

,left_shoulder_x,left_shoulder_y,left_shoulder_z,left_shoulder_vis,right_shoulder_x,right_shoulder_y,right_shoulder_z,right_shoulder_vis,left_elbow_x,left_elbow_y,...,right_knee_angle,left_knee_angle,left_shank_angle,right_shank_angle,left_ankle_change,right_ankle_change,left_ankle_hip_distance,right_ankle_hip_distance,left_lean_angle,right_lean_angle
0,0.534510,0.446887,-0.429476,1.000000,0.492759,0.453778,0.248402,0.999686,0.655496,0.507027,...,123.363131,154.164353,7.045282,-69.641411,0.003862,-0.005471,0.252633,0.268873,10.304059,0.023489
1,0.534547,0.450484,-0.436400,1.000000,0.492668,0.455598,0.248885,0.999679,0.647803,0.518851,...,105.949361,147.178645,-2.156940,-73.479725,0.002093,-0.004736,0.245270,0.255510,10.327064,0.592572
2,0.527605,0.459486,-0.524648,1.000000,0.492851,0.461270,0.179419,0.999630,0.626247,0.537474,...,95.630032,133.838887,-15.548759,-75.033486,-0.001234,-0.003009,0.229546,0.228749,7.842514,1.461519
3,0.521322,0.467924,-0.585742,1.000000,0.484806,0.466932,0.145372,0.999639,0.599499,0.549772,...,83.442911,114.381250,-33.393563,-74.677457,-0.001585,-0.001398,0.219003,0.200489,4.356775,0.951518
4,0.517395,0.467651,-0.556952,0.999999,0.484626,0.470058,0.172725,0.999508,0.567996,0.556933,...,64.119825,108.943128,-41.841483,-76.112785,-0.000201,0.002694,0.216548,0.176047,0.506180,1.032170


In [53]:
# Add a new column 'right_knee_angle' using calculate_angle function
df_dup['right_knee_angle'] = calculate_angle(df_dup, 'right_hip', 'right_knee', 'right_ankle')

# Add a new column 'left_knee_angle' using calculate_angle function
df_dup['left_knee_angle'] = calculate_angle(df_dup, 'left_hip', 'left_knee', 'left_ankle')

In [54]:
df_dup['left_shank_angle'] = calculate_shank_angle(df_dup, 'left_ankle', 'left_knee')
df_dup['right_shank_angle'] = calculate_shank_angle(df_dup, 'right_ankle', 'right_knee')

In [55]:
df_dup['left_ankle_change'] = np.gradient(df_dup['left_ankle_y'])
df_dup['right_ankle_change'] = np.gradient(df_dup['right_ankle_y'])

In [56]:
# Calculate relative distance between left ankle and left hip
df_dup['left_ankle_hip_distance'] = np.sqrt((df_dup['left_ankle_x'] - df_dup['left_hip_x']) ** 2 + 
                                      (df_dup['left_ankle_y'] - df_dup['left_hip_y']) ** 2)

df_dup['right_ankle_hip_distance'] = np.sqrt((df_dup['right_ankle_x'] - df_dup['right_hip_x']) ** 2 + 
                                      (df_dup['right_ankle_y'] - df_dup['right_hip_y']) ** 2)

In [57]:
df_dup['left_lean_angle'] = calculate_lean_angle(df_dup, 'left_shoulder', 'left_hip')
df_dup['right_lean_angle'] = calculate_lean_angle(df_dup, 'right_shoulder', 'right_hip')

In [58]:
df_dup.shape

(258, 98)

In [59]:
df_dup.columns

Index(['left_shoulder_x', 'left_shoulder_y', 'left_shoulder_z',
       'left_shoulder_vis', 'right_shoulder_x', 'right_shoulder_y',
       'right_shoulder_z', 'right_shoulder_vis', 'left_elbow_x',
       'left_elbow_y', 'left_elbow_z', 'left_elbow_vis', 'right_elbow_x',
       'right_elbow_y', 'right_elbow_z', 'right_elbow_vis', 'left_wrist_x',
       'left_wrist_y', 'left_wrist_z', 'left_wrist_vis', 'right_wrist_x',
       'right_wrist_y', 'right_wrist_z', 'right_wrist_vis', 'left_pinky_x',
       'left_pinky_y', 'left_pinky_z', 'left_pinky_vis', 'right_pinky_x',
       'right_pinky_y', 'right_pinky_z', 'right_pinky_vis', 'left_index_x',
       'left_index_y', 'left_index_z', 'left_index_vis', 'right_index_x',
       'right_index_y', 'right_index_z', 'right_index_vis', 'left_thumb_x',
       'left_thumb_y', 'left_thumb_z', 'left_thumb_vis', 'right_thumb_x',
       'right_thumb_y', 'right_thumb_z', 'right_thumb_vis', 'left_hip_x',
       'left_hip_y', 'left_hip_z', 'left_hip_vis', 'rig

In [73]:
with open('../models/scaler.pkl', 'rb') as f:
    ss = pickle.load(f)

In [74]:
X = ss.transform(df_dup)

In [76]:
# Reshape data to have 1 sample, 258 frames and 98 features
X_pred = np.expand_dims(X, axis=0)

In [77]:
X_pred.shape

(1, 258, 98)

In [78]:
predictions = model.predict(X_pred)

1/1 [==============================] - 0s 21ms/step


In [79]:
# Get the max probability across all 258 frames within each video
video_max_probabilities = np.max(predictions, axis = 1)

In [80]:
video_max_probabilities

array([[0.99881095]], dtype=float32)

In [81]:
threshold = 0.7

y_pred = (video_max_probabilities > threshold).astype(int) # Convert all labels to binary

# Flatten y_pred into a 1D array
y_pred_flat = np.array(y_pred).flatten()

if y_pred_flat == 1:
    print('Runner is overstriding')
else:
    print('Runner is not overstriding')

Runner is overstriding


In [66]:
max_left_ankle_y_frame = df_dup['left_ankle_y'].idxmax()

In [67]:
max_left_ankle_y_frame

158

In [68]:
max_left_ankle_y_frame = np.argmax(df_dup['left_ankle_y'].values)

In [ ]:
import streamlit as st
import mediapipe as mp
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
import tempfile
import os

tf.__version__ = "2.13.0"
mp.__version__ = "0.10.9"

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, 
                              results.pose_landmarks, 
                              mp_pose.POSE_CONNECTIONS)

def duplicate_frames_2(df, max_frames):
    num_dup = max_frames - len(df)

    if num_dup > 0:
        # Duplicate frames up to the required number of duplicates
        dup_frames = df.loc[:num_dup - 1].copy()
        df = pd.concat([df, dup_frames], ignore_index=True)
    elif num_dup < 0:
        # Truncate DataFrame if it exceeds max_frames
        df = df.iloc[:max_frames]

    return df

def calculate_angle(df, a, b, c):
    a_x = a + '_x'
    a_y = a + '_y'
    b_x = b + '_x'
    b_y = b + '_y'
    c_x = c + '_x'
    c_y = c + '_y'

    radians = np.arctan2(df[c_y].values - df[b_y].values, df[c_x].values - df[b_x].values) - \
              np.arctan2(df[a_y].values - df[b_y].values, df[a_x].values - df[b_x].values)
    angles = np.abs(radians * 180.0 / np.pi)
    angles = np.where(angles > 180.0, 360 - angles, angles)
    return angles

def calculate_shank_angle(df, ankle, knee):
    ankle_x = ankle + '_x'
    ankle_y = ankle + '_y'
    knee_x = knee + '_x'
    knee_y = knee + '_y'

    angle_rad = np.arctan2(df[knee_y] - df[ankle_y], df[knee_x] - df[ankle_x])
    angle_deg = np.degrees(angle_rad) + 90

    return angle_deg

def calculate_lean_angle(df, a, b):
    a_x = a + '_x'
    a_y = a + '_y'
    b_x = b + '_x'
    b_y = b + '_y'

    radians = np.arctan2(0 - df[b_y].values, df[b_x].values - df[b_x].values) - \
              np.arctan2(df[a_y].values - df[b_y].values, df[a_x].values - df[b_x].values)
    angles = np.abs(radians * 180.0 / np.pi)
    angles = np.where(angles > 180.0, 360 - angles, angles)
    return angles

# Load the pre-trained Keras model
model = load_model('../models/cnn_lstm_model.h5')

mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.7, min_tracking_confidence=0.7)
mp_drawing = mp.solutions.drawing_utils # Draws the points and lines between points

st.title("Runner Overstriding Detection")

uploaded_file = st.file_uploader("Upload a video", type=["mp4"])

if uploaded_file is not None:
    # Save the uploaded video to a temporary file
    with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
        tmp_file.write(uploaded_file.read())
        video_path = tmp_file.name

    # Use VideoCapture to read frames directly from the uploaded video file
    video_capture = cv2.VideoCapture(video_path)
    if video_capture.isOpened():
        ret, frame = video_capture.read()
        if ret:
            # Process the frame further and perform analysis
            image, results = mediapipe_detection(frame, pose)
            keypoints = results.pose_landmarks.landmark
            keypoints_noface = keypoints[11:]
            st.write(keypoints_noface)

            features = []
            for keypoint in keypoints_noface:
                feature_list = [keypoint.x, keypoint.y, keypoint.z, keypoint.visibility]
                features.append(feature_list)

            column_names = ['left_shoulder_x', 'left_shoulder_y', 'left_shoulder_z', 'left_shoulder_vis', 
                            'right_shoulder_x', 'right_shoulder_y', 'right_shoulder_z', 'right_shoulder_vis', 
                            'left_elbow_x', 'left_elbow_y', 'left_elbow_z', 'left_elbow_vis', 
                            'right_elbow_x', 'right_elbow_y', 'right_elbow_z', 'right_elbow_vis', 
                            'left_wrist_x', 'left_wrist_y', 'left_wrist_z', 'left_wrist_vis', 
                            'right_wrist_x', 'right_wrist_y', 'right_wrist_z', 'right_wrist_vis', 
                            'left_pinky_x', 'left_pinky_y', 'left_pinky_z', 'left_pinky_vis', 
                            'right_pinky_x', 'right_pinky_y', 'right_pinky_z', 'right_pinky_vis', 
                            'left_index_x', 'left_index_y', 'left_index_z', 'left_index_vis', 
                            'right_index_x', 'right_index_y', 'right_index_z', 'right_index_vis', 
                            'left_thumb_x', 'left_thumb_y', 'left_thumb_z', 'left_thumb_vis', 
                            'right_thumb_x', 'right_thumb_y', 'right_thumb_z', 'right_thumb_vis', 
                            'left_hip_x', 'left_hip_y', 'left_hip_z', 'left_hip_vis', 
                            'right_hip_x', 'right_hip_y', 'right_hip_z', 'right_hip_vis', 
                            'left_knee_x', 'left_knee_y', 'left_knee_z', 'left_knee_vis', 
                            'right_knee_x', 'right_knee_y', 'right_knee_z', 'right_knee_vis', 
                            'left_ankle_x', 'left_ankle_y', 'left_ankle_z', 'left_ankle_vis', 
                            'right_ankle_x', 'right_ankle_y', 'right_ankle_z', 'right_ankle_vis', 
                            'left_heel_x', 'left_heel_y', 'left_heel_z', 'left_heel_vis', 
                            'right_heel_x', 'right_heel_y', 'right_heel_z', 'right_heel_vis', 
                            'left_foot_index_x', 'left_foot_index_y', 'left_foot_index_z', 'left_foot_index_vis',
                             'right_foot_index_x', 'right_foot_index_y', 'right_foot_index_z', 'right_foot_index_vis']

            df = pd.DataFrame(features, columns=column_names)

            df_dup = duplicate_frames_2(df, 258)

            df_dup['right_knee_angle'] = calculate_angle(df_dup, 'right_hip', 'right_knee', 'right_ankle')
            df_dup['left_knee_angle'] = calculate_angle(df_dup, 'left_hip', 'left_knee', 'left_ankle')
            df_dup['left_shank_angle'] = calculate_shank_angle(df_dup, 'left_ankle', 'left_knee')
            df_dup['right_shank_angle'] = calculate_shank_angle(df_dup, 'right_ankle', 'right_knee')
            df_dup['left_ankle_change'] = np.gradient(df_dup['left_ankle_y'])
            df_dup['right_ankle_change'] = np.gradient(df_dup['right_ankle_y'])
            df_dup['left_ankle_hip_distance'] = np.sqrt((df_dup['left_ankle_x'] - df_dup['left_hip_x']) ** 2 + 
                                      (df_dup['left_ankle_y'] - df_dup['left_hip_y']) ** 2)
            df_dup['right_ankle_hip_distance'] = np.sqrt((df_dup['right_ankle_x'] - df_dup['right_hip_x']) ** 2 + 
                                      (df_dup['right_ankle_y'] - df_dup['right_hip_y']) ** 2)
            df_dup['left_lean_angle'] = calculate_lean_angle(df_dup, 'left_shoulder', 'left_hip')
            df_dup['right_lean_angle'] = calculate_lean_angle(df_dup, 'right_shoulder', 'right_hip')

            # Reshape data for model input
            X_pred = np.expand_dims(df_dup.values, axis=0)

            # Make predictions with the model
            predictions = model.predict(X_pred)
            max_probabilities = np.max(predictions, axis=1)
            threshold = 0.7
            labels = (max_probabilities > threshold).astype(int)
            flat_labels = np.array(labels).flatten()

            if flat_labels == 1:
                st.write('Runner is overstriding')
            else:
                st.write('Runner is not overstriding')

            # Calculate frame indices based on conditions
            max_left_ankle_y_frame = np.argmax(df_dup['left_ankle_y'].values)
            # Debugging statements
            st.write(f"DataFrame Info:")
            st.write(df_dup.head())  # Check the first few rows of the DataFrame
            st.write(f"Left Ankle Y values:")
            st.write(df_dup['left_ankle_y'].values)  # Check the values of 'left_ankle_y' column
            st.write(f"Argmax Left Ankle Y:")
            st.write(np.argmax(df_dup['left_ankle_y'].values))  # Check the argmax value

            # Display frames with pose skeleton based on conditions
            video_capture.set(cv2.CAP_PROP_POS_FRAMES, max_left_ankle_y_frame)
            ret, frame = video_capture.read()
            if ret:
                image, results = mediapipe_detection(frame, pose)
                image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                draw_landmarks(image_rgb, results)
                st.image(image_rgb, channels="RGB", caption="Frame with maximum left ankle y")

        else:
            st.error("Failed to read the uploaded video frame.")
    else:
        st.error("Failed to open the uploaded video file.")
else:
    st.warning("Please upload an MP4 video.")   